In [2]:
from requests.auth import HTTPBasicAuth
from impala.dbapi import connect
from datetime import datetime
from pyhive import hive
import pandas as pd
import requests
import sqlparse
import json
import sys
import os
import re

In [3]:
'''Variables'''
## Atlas dev
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}
credentials = HTTPBasicAuth('atlasadmin', 'xNB8lZ!Dut')
atlas_urls = ["172.30.213.141","172.30.213.142"]
atlas_port = 31000

## Hive dev
hive_host = 'edh-master-01d.root.corp'
hive_port = 10000
hive_user = 'admin'


## Impala dev
impala_host = '172.30.213.211'
impala_port = 21050
impala_user = 'admin'

In [4]:
#HIVE
def describe_table( hive_database, table_name ):
    #Creo la conexión y obtengo el cursos
    hive_conn = hive.connect(host=hive_host, port=hive_port, username=hive_user, database=hive_database)
    hive_cursor = hive_conn.cursor()

    # Obtengo el create
    hive_cursor.execute(f'DESCRIBE {hive_database}.{table_name}')
    databases = hive_cursor.fetchall()
    
    return databases

In [5]:
with open('data/json_completo.json') as f:
    file = f.read()
    json_file = json.loads(file)

In [ ]:
# Print json completo
print(json.dumps(json_file, indent=2))

In [ ]:
-- Así es el formato de cada registro en el JSON
"de_bsj_1raw.sfb_aampr": {
    "database": "de_bsj_1raw",
    "table_name": "sfb_aampr",
    "create_table": "CREATE EXTERNAL TABLE `de_bsj_1raw`.`sfb_aampr`(   `glb_dtime` string COMMENT '',    `aco_conce` string COMMENT 'C\u00f3digo de concepto',    `aco_produ` string COMMENT 'C\u00f3digo de producto',    `acoidprod` string COMMENT 'Identificaci\u00f3n del producto',    `ase_essfb` string COMMENT 'Fecha de vencimiento del producto',    `aseprocom` string COMMENT 'Fecha de vigencia del producto',    `dco_moned` string COMMENT 'Nombre del producto',    `afevenpro` string COMMENT 'Nro. de Campo c.467',    `afevigpro` string COMMENT 'Nro. de Campo gtia.',    `ano_produ` string COMMENT 'Nro. de Campo lisol',    `anu_ca467` string COMMENT 'Factor de ponderaci\u00f3n',    `anu_campo` string COMMENT 'Fecha de vencimiento del producto en formato relativa',    `anu_lisol` string COMMENT 'Fecha de vigencia del producto en formato relativa',    `apofacpon` string COMMENT 'Se\u00f1al que indica si es un producto SFB',    `arevenpro` string COMMENT 'Se\u00f1al de si Controla productos privados',    `arevigpro` string COMMENT 'Se\u00f1al es producto cta comitente',    `estado` string COMMENT 'C\u00f3digo de moneda',    `ase_priva` string COMMENT 'Estado') COMMENT 'MAESTRO DE PRODUCTOS GENERAL' PARTITIONED BY (    `fecha_proceso` varchar(8)) ROW FORMAT SERDE    'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'  STORED AS INPUTFORMAT    'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat'  OUTPUTFORMAT    'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat' LOCATION   'hdfs://nameservice1/user/admin/dev/bsj/01-raw/sfb/productos/plazo_fijo/sfb_aampr' ",
    "associated_term": "aampr",
    "loading_query": null,
    "affected_tables": [
      "de_bsj_2cur.productos_aampr"
    ]
  }

In [7]:
options = ['# Partition Information', '# col_name', '']

def es_garbage( campo, opciones ):
    return campo in opciones

In [9]:
pares_raw_cur = []

for table_raw in json_file.keys():
    if '1raw' in table_raw:
        db_raw, table_name_raw = table_raw.split('.')
        # Obtenemos lista de tablas afectadas
        list_affected_tables = json_file[table_raw]['affected_tables']
        # Si afecta más de una tabla, debemos obtener solo la que posea la misma fuente en el nombre en curado
        if len(list_affected_tables)>1:
            source = table_name_raw.split('_')[-1]
            # Filtra la tabla que contiene la fuente
            cur_table = [t for t in list_affected_tables if source in t]
            cur_table = cur_table[0]
        if len(list_affected_tables)>0:
            # Si tiene 1 solo elemento, entonces esa es la tabla en curado
            cur_table = list_affected_tables[0]
        
        # Si llgué acá es porque tengo la tabla en raw + cur
        db_cur, table_name_cur = cur_table.split('.')

        create_raw = describe_table(db_raw, table_name_raw) 
        create_cur = describe_table(db_cur, table_name_cur) 
        if len(create_raw) == len(create_cur):
            #La cantidad de campos es igual
            #Zippeamos campo (raw, cur)
            pares_raw_cur.append( [(f'{table_raw}.{tupla_raw[0]}', f'{cur_table}.{tupla_cur[0]}') for tupla_raw, tupla_cur in zip(create_raw, create_cur) if not es_garbage(tupla_raw[0], options)] )

In [10]:
# Aplanar la lista anidada
lista_aplanada = [tupla for sublist in pares_raw_cur for tupla in sublist]

# Crear un DataFrame a partir de la lista aplanada
df = pd.DataFrame(lista_aplanada, columns=['nombre_campo_raw', 'nombre_campo_cur'])

In [12]:
df.to_csv('data_procesada/mapeo_campos_raw_cur.csv', sep='|', index=False)